### **Import necesary tools**

In [70]:
from transformers import pipeline, set_seed

In [71]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

responses = generator("Inima imi spune", max_length=128, num_return_sequences=5)

for response in responses:
  print(response["generated_text"])
  print("-----")

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Inima imi spune, or "the city of the gods," is one such city. It was a large city, and in its name was the city of the gods. The worship of the gods was done in the name of the god, and the city was called the "City of the Gods." The worship of the gods was done by the priests and priests of the city.

The "City of the Gods" was also known as the "City of the Gods" or "City of the Gods." According to the New Testament, the "City of the Gods" was a capital city of the Babylonians, who ruled over the entire world. The "City of the Gods" was also a capital city of the Assyrians, who ruled over the entire Earth. The "City of the Gods" was also known as the "City of the Gods." According to the New Testament, the "City of the Gods" was a capital city of the Babylonians, who ruled over the entire Earth. The "City of the Gods" was also known as the "City of the Gods." According to the New Testament, the "City of the Gods" was a capital city of the Babylonians, who ruled over the entire Earth.


In [72]:
base_model = "readerbench/RoGPT2-medium"  # or RoGPT2-base / RoGPT2-large

### **Reformat the dataset as poem-level text**

In [73]:
import re
from datasets import load_dataset, Dataset

raw = load_dataset("json", data_files={"train": "results.jsonl"})["train"]
df = raw.to_pandas()

# Sort to reconstruct poems correctly
df = df.sort_values(["title", "verse_index"])

def normalize_verse(v: str):
    v = v.replace("\r\n", "\n").strip()
    # treat '@' as stanza boundary marker (often appears as " @" at end)
    stanza_break = "@" in v
    v = v.replace("@", "").strip()
    return v, stanza_break

poems = []
for title, g in df.groupby("title", sort=False):
    lines = [f"<|title|> {title}"]  # title conditioning (recommended)
    for v in g["verse"].tolist():
        line, br = normalize_verse(v)
        if line:
            lines.append(line)
        if br:
            lines.append("")  # blank line = stanza separator
    poem_text = "\n".join(lines).strip()
    poems.append(poem_text)

poem_ds = Dataset.from_dict({"text": poems})
print(poem_ds[0]["text"][:300])
print("Num poems:", len(poem_ds))

<|title|> ***
Ce fel de tren marfar ești tu
dacă ți-e trupul meu șină de carne;
Ce fel de măr ești tu
dacă ți-e ramură viața mea?

Eu locuiesc într-un tril
de privighetoare
Dorm cu ceafa pe nota Do
și-mi încălț piciorul
într-un saxofon

Du-te, îmi strigă ciocanul,
du-te,
du-te idiotule de cui de fie
Num poems: 139


### **Import Tokenizer**

In [74]:
from transformers import AutoTokenizer

In [75]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Set the padding token to match the end-of-sequence (EOS) token
tokenizer.pad_token = tokenizer.eos_token

In [76]:
# Add a special token for title to make prompting stable
specials = {"additional_special_tokens": ["<|title|>"]}
num_added = tokenizer.add_special_tokens(specials)

In [77]:
def preprocess(examples):
    # Add EOS to mark poem end (as in the article) :contentReference[oaicite:6]{index=6}
    return {"text": [t.strip() + tokenizer.eos_token for t in examples["text"]]}

def tokenize_fn(examples):
    return tokenizer(examples["text"], truncation=False, add_special_tokens=True)

In [78]:
poem_ds = poem_ds.map(preprocess, batched=True)
tok = poem_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

block_size = 256  # 512 also fine; 256 is easier on memory
def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_len = len(concatenated["input_ids"])
    total_len = (total_len // block_size) * block_size
    return {
        k: [t[i:i+block_size] for i in range(0, total_len, block_size)]
        for k, t in concatenated.items()
    }

lm_ds = tok.map(group_texts, batched=True, batch_size=1000)

Map: 100%|██████████| 139/139 [00:00<00:00, 5566.77 examples/s]


#### **Train with Trainer**

In [79]:
import torch
from transformers import (
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)

model = AutoModelForCausalLM.from_pretrained(base_model)

# If we added tokens (e.g., <|title|>), resize embeddings
if num_added > 0:
    model.resize_token_embeddings(len(tokenizer))

model.config.bos_token_id = tokenizer.bos_token_id or tokenizer.eos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.eos_token_id

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # causal LM (as in the article) :contentReference[oaicite:7]{index=7}
)

# Optional: split train/eval for perplexity tracking
split = lm_ds.train_test_split(test_size=0.05, seed=42)
train_ds, eval_ds = split["train"], split["test"]

use_fp16 = torch.cuda.is_available()

training_args = TrainingArguments(
    output_dir="./nichita-Ro-gpt2",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    save_strategy="epoch",
    logging_steps=50,
    eval_strategy="no",
    fp16=torch.cuda.is_available(),
    report_to="none",
    dataloader_num_workers=0,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
)

trainer.train()

model.save_pretrained("./nichita-Ro-gpt2", safe_serialization=True)
tokenizer.save_pretrained("./nichita-Ro-gpt2")

Step,Training Loss


('./nichita-Ro-gpt2\\tokenizer_config.json',
 './nichita-Ro-gpt2\\special_tokens_map.json',
 './nichita-Ro-gpt2\\vocab.json',
 './nichita-Ro-gpt2\\merges.txt',
 './nichita-Ro-gpt2\\added_tokens.json',
 './nichita-Ro-gpt2\\tokenizer.json')

In [91]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_DIR = "./nichita-Ro-gpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# IMPORTANT for GPT-2 style models
tokenizer.pad_token = tokenizer.eos_token

def generate(prompt, max_new_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.6,
            top_p=0.95,
            top_k=50,
            repetition_penalty=1.15,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out[0], skip_special_tokens=False)

In [92]:
print(generate("Poetul si soldatul\n"))

Poetul si soldatul
Din zori și până-n seară, cu toții suntem eroi. ___________ Cu fruntea sus și cu pieptul în vânt. ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ────


Poetul si soldatul
Din zori și până-n seară, cu toții suntem eroi. ___________ Cu fruntea sus și cu pieptul în vânt. ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ────

In [ ]:
print(generate("Inima ma doare\n"))

Inima ma doare
Iarăși am plecat din viață,֔de parcă aș fi făcut parte din nou din viață.─┤ O lacrimă se rostogolește──O altă lacrimă se rostogolte──Și-n fiecare lacrimă se oglindește aceeași speranță志<|endoftext|>

In [94]:
print(generate("Ganduri de seara\n"))

Ganduri de seara
-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────-O mie și una de nopți───────────────
